In [36]:
import numpy as np
import opendssdirect as dss
from utils.device.Inverter import Inverter
from utils.controller.AdaptiveInvController import AdaptiveInvController
from utils.controller.FixedInvController import FixedInvController
import matplotlib.pyplot as plt
from math import tan,acos
import copy
import pandas as pd
import time

%matplotlib inline

# import numpy as np
# from matplotlib.mlab import PCA
from sklearn.decomposition import PCA

import scipy.linalg as spla


#######################################################
#######################################################
###Global variable initialization and error checking###
#######################################################
#######################################################
Sbase=1
LoadScalingFactor = 3
GenerationScalingFactor = 5
SlackBusVoltage = 1.00
NoiseMultiplyer= 1
#Set simulation analysis period - the simulation is from StartTime to EndTime
StartTime = 40600
EndTime = StartTime + 640
EndTime += 1 # creating a list, last element does not count, so we increase EndTime by 1
#Set hack parameters
TimeStepOfHack = 160
PercentHacked = np.array([0,0,0,0,0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0,0,0,0,0,0,0,0,0,0,0,0,0])

#Set initial VBP parameters for uncompromised inverters
VBP_normal=np.array([0.97, 1, 1, 1.03, 1.06])
VBP_attack=np.array([0.997, 1, 1, 1.003, 1.007])

#Set delays for each node
Delay_VBPCurveShift = (30+2*np.random.randn(31)).astype(int)
#Delay_VBPCurveShift = (10+2*np.random.randn(31)).astype(int)
lpf_meas_vector = (1+0.2*np.random.randn(31))
lpf_output_vector= (0.1+0.015*np.random.randn(31))

#Set observer voltage threshold
ThreshHold_vqvp = 0.06
adaptive_gain=800


power_factor=0.9
pf_converted=tan(acos(power_factor))
# Number_of_Inverters = 13 #even feeder is 34Bus, we only have 13 inverters


#Error checking of the global variable -- TODO: add error handling here!
if EndTime < StartTime or EndTime < 0 or StartTime < 0:
    print('Setup Simulation Times Inappropriately.')
if NoiseMultiplyer < 0:
    print('Setup Noise Multiplyer Correctly.')
    
dss.run_command('Redirect feeder/03node_multiphase_balanced.dss')
# dss.run_command('Redirect feeder/feeder34_B_NR/network_05node_3phase_radial.dss')

dss.Solution.Solve()
if not dss.Solution.Converged:
    print('Initial Solution Not Converged. Check Model for Convergence')
else:
    print('Initial Model Converged. Proceeding to Next Step.')
    #Doing this solve command is required for GridPV, that is why the monitors
    #go under a reset process
    dss.Monitors.ResetAll()
    
    #set solution Params
    #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
    dss.Solution.Mode(1)
    dss.Solution.Number(1)
    dss.Solution.StepSize(1)
    dss.Solution.ControlMode(-1)
    dss.Solution.MaxControlIterations(1000000)
    dss.Solution.MaxIterations(30000)
    #Easy process to get all names and count of loads, a trick to avoid
    #some more lines of code
    TotalLoads=dss.Loads.Count()
    AllLoadNames=dss.Loads.AllNames()
    print('OpenDSS Model Compliation Done.')

print('')
print(TotalLoads)
# dir(dss.Loads)

# (dss.Loads.AllNames())

# [x for x in dss.Circuit.AllBusNames() if 'bus' in x ]

# dss.Circuit.AllBusNames()

# dir(dss.Loads.AllNames)
print(len(dss.Loads.AllNames()))
print(dss.Loads.AllNames())

print(len(dss.Circuit.AllBusNames()))
print(dss.Circuit.AllBusNames())

print(dss.Bus.kVBase())

for k1 in range(len(dss.Circuit.AllBusNames())):
    if dss.Circuit.AllBusNames()[k1] == 'sourcebus':
        dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
#         print(dss.Bus.kVBase())
        tempVoltage = dss.Bus.VMagAngle()
        tempVoltage = tempVoltage
    #     print(tempVoltage)
#         print(tempVoltage[::2])
#         print(tempVoltage[0]*np.sqrt(3)/1)
        
        print(dss.Circuit.AllBusNames()[k1])
        print('BASE - LN')
        print(dss.Bus.kVBase())
        print('VMAGANGLE - LN POLAR')
        print(dss.Bus.VMagAngle())
        print('puVMAGANGLE - LN POLAR')
        print(dss.Bus.puVmagAngle())
        print('VOTLAGES - LN CARTESIAN')
        print(dss.Bus.Voltages())
        print('puVOTLAGES - LN CARTESIAN')
        print(dss.Bus.PuVoltage())
        print('VLL - LL CARTESIAN')
        print(dss.Bus.VLL())
        print('puVLL - LL CARTESIAN')
        print(dss.Bus.puVLL())
        
        tempvoltage = dss.Bus.Voltages()
        
        Vab = tempvoltage[0]
        
print('~~~~~')
        
for k1 in range(len(dss.Circuit.AllBusNames())):
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
    print(dss.Circuit.AllBusNames()[k1])
    print('BASE - LN')
    print(dss.Bus.kVBase())
    print('VMAGANGLE - LN POLAR')
    print(dss.Bus.VMagAngle())
    print('puVMAGANGLE - LN POLAR')
    print(dss.Bus.puVmagAngle())
    
#     print('puVLL - LL CARTESIAN')
#     print(dss.Bus.puVLL())
#     tempVoltage = np.array(dss.Bus.puVLL())#.astype('float')
#     tempMag = np.array([np.sqrt(tempVoltage[0]**2 + tempVoltage[1]**2), \
#                         np.sqrt(tempVoltage[2]**2 + tempVoltage[3]**2), \
#                         np.sqrt(tempVoltage[4]**2 + tempVoltage[5]**2)])
#     print(tempMag)
#     tempAngle = np.array([np.angle(tempVoltage[0] + 1j*tempVoltage[1], deg=True), \
#                          np.angle(tempVoltage[2] + 1j*tempVoltage[3], deg=True)+360, \
#                          np.angle(tempVoltage[4] + 1j*tempVoltage[5], deg=True)])
#     print(tempAngle)
    print('')

    
# dir(dss.Bus)

Initial Model Converged. Proceeding to Next Step.
OpenDSS Model Compliation Done.

6
6
['s701a', 's701b', 's701c', 's702a', 's702b', 's702c']
5
['sourcebus', '799', '799r', '701', '702']
132.79056191361394
sourcebus
BASE - LN
132.79056191361394
VMAGANGLE - LN POLAR
[132790.32487668263, -0.0001990962574346758, 132790.32022718244, -120.00019722632115, 132790.32043358576, 119.9998030146867]
puVMAGANGLE - LN POLAR
[0.9999982149564858, -0.0001990962574346758, 0.9999981799427007, -120.00019722632115, 0.999998181497053, 119.9998030146867]
VOTLAGES - LN CARTESIAN
[132790.32487588093, -0.461431137380978, -66395.55598400848, -114999.56213661014, -66394.76485518896, 115000.0191333834]
puVOTLAGES - LN CARTESIAN
[0.9999982149504485, -3.47487901799195e-06, -0.5000020711351586, -0.8660221063860125, -0.49999611341641625, 0.8660255478713611]
VLL - LL CARTESIAN
[199185.8808598894, 114999.10070547275, -0.7911288195173256, -229999.58126999353, -199185.0897310699, 115000.48056452078]
puVLL - LL CARTESIAN
[

In [37]:
print(dss.RegControls.AllNames())

for k1 in range(len(dss.RegControls.AllNames())):
    print(dss.RegControls.AllNames()[k1])
    dss.RegControls.Name(dss.RegControls.AllNames()[k1])
    
    print(dss.RegControls.TapNumber())
    

dss.RegControls.Name('reg1')
dss.RegControls.Name()

for k1 in range(len(dss.RegControls.AllNames())):
    print(dss.RegControls.AllNames()[k1])
    dss.RegControls.Name(dss.RegControls.AllNames()[k1])
    
    dss.RegControls.TapNumber(0)    
    print(dss.RegControls.TapNumber())


['reg1']
reg1
1
reg1
0


In [27]:
dss.Solution.Solve()
if not dss.Solution.Converged:
    print('Initial Solution Not Converged. Check Model for Convergence')
else:
    print('Initial Model Converged. Proceeding to Next Step.')
    #Doing this solve command is required for GridPV, that is why the monitors
    #go under a reset process
    dss.Monitors.ResetAll()
    
    #set solution Params
    #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
    dss.Solution.Mode(1)
    dss.Solution.Number(1)
    dss.Solution.StepSize(1)
    dss.Solution.ControlMode(-1)
    dss.Solution.MaxControlIterations(1000000)
    dss.Solution.MaxIterations(30000)
    #Easy process to get all names and count of loads, a trick to avoid
    #some more lines of code
    TotalLoads=dss.Loads.Count()
    AllLoadNames=dss.Loads.AllNames()
    print('OpenDSS Model Compliation Done.')

print('~~~~~')
        
for k1 in range(len(dss.Circuit.AllBusNames())):
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
    print(dss.Circuit.AllBusNames()[k1])
    print('BASE - LN')
    print(dss.Bus.kVBase())
    print('VMAGANGLE - LN POLAR')
    print(dss.Bus.VMagAngle())
    print('puVMAGANGLE - LN POLAR')
    print(dss.Bus.puVmagAngle())
    print('')

print('')
print(dss.RegControls.AllNames())

for k1 in range(len(dss.RegControls.AllNames())):
    print(dss.RegControls.AllNames()[k1])
    dss.RegControls.Name(dss.RegControls.AllNames()[k1])
    
    print(dss.RegControls.TapNumber())

Initial Model Converged. Proceeding to Next Step.
OpenDSS Model Compliation Done.
~~~~~
sourcebus
BASE - LN
132.79056191361394
VMAGANGLE - LN POLAR
[132790.32819749997, -0.00019646345805470616, 132790.32361287173, -120.00019462107957, 132790.3238160837, 119.9998056162356]
puVMAGANGLE - LN POLAR
[0.9999982399644177, -0.00019646345805470616, 0.999998205439161, -120.00019462107957, 0.9999982069694803, 119.9998056162356]

799
BASE - LN
2.7712812921102037
VMAGANGLE - LN POLAR
[2739.8645999463756, -1.3117032476120123, 2739.1616821441053, -121.29837444829063, 2739.161686325877, 118.7016257886502]
puVMAGANGLE - LN POLAR
[0.988663477701354, -1.3117032476120123, 0.9884098340873796, -121.29837444829063, 0.9884098355963464, 118.7016257886502]

799r
BASE - LN
2.7712812921102037
VMAGANGLE - LN POLAR
[2739.8309024712335, -1.3121647314039884, 2739.1278224874604, -121.29882502851235, 2739.1278266691697, 118.70117520842838]
puVMAGANGLE - LN POLAR
[0.9886513181723887, -1.3121647314039884, 0.9883976160362

In [12]:
for k1 in range(len(dss.RegControls.AllNames())):
    print(dss.RegControls.AllNames()[k1])
    dss.RegControls.Name(dss.RegControls.AllNames()[k1])
    
    dss.RegControls.Reset()    
    print(dss.RegControls.TapNumber())

dss.Solution.Solve()
if not dss.Solution.Converged:
    print('Initial Solution Not Converged. Check Model for Convergence')
else:
    print('Initial Model Converged. Proceeding to Next Step.')
    #Doing this solve command is required for GridPV, that is why the monitors
    #go under a reset process
    dss.Monitors.ResetAll()
    
    #set solution Params
    #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
    dss.Solution.Mode(1)
    dss.Solution.Number(1)
    dss.Solution.StepSize(1)
    dss.Solution.ControlMode(-1)
    dss.Solution.MaxControlIterations(1000000)
    dss.Solution.MaxIterations(30000)
    #Easy process to get all names and count of loads, a trick to avoid
    #some more lines of code
    TotalLoads=dss.Loads.Count()
    AllLoadNames=dss.Loads.AllNames()
    print('OpenDSS Model Compliation Done.')

print('~~~~~')
        
for k1 in range(len(dss.Circuit.AllBusNames())):
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
    print(dss.Circuit.AllBusNames()[k1])
    print('BASE - LN')
    print(dss.Bus.kVBase())
    print('VMAGANGLE - LN POLAR')
    print(dss.Bus.VMagAngle())
    print('puVMAGANGLE - LN POLAR')
    print(dss.Bus.puVmagAngle())
    print('')

print('')
print(dss.RegControls.AllNames())

for k1 in range(len(dss.RegControls.AllNames())):
    print(dss.RegControls.AllNames()[k1])
    dss.RegControls.Name(dss.RegControls.AllNames()[k1])
    
    print(dss.RegControls.TapNumber())

reg1
0
Initial Model Converged. Proceeding to Next Step.
OpenDSS Model Compliation Done.
~~~~~
sourcebus
BASE - LN
132.79056191361394
VMAGANGLE - LN POLAR
[132790.32819749997, -0.00019646345805470616, 132790.32361287173, -120.00019462107957, 132790.3238160837, 119.9998056162356]
puVMAGANGLE - LN POLAR
[0.9999982399644177, -0.00019646345805470616, 0.999998205439161, -120.00019462107957, 0.9999982069694803, 119.9998056162356]

799
BASE - LN
2.7712812921102037
VMAGANGLE - LN POLAR
[2739.8645999463756, -1.3117032476120123, 2739.1616821441053, -121.29837444829063, 2739.161686325877, 118.7016257886502]
puVMAGANGLE - LN POLAR
[0.988663477701354, -1.3117032476120123, 0.9884098340873796, -121.29837444829063, 0.9884098355963464, 118.7016257886502]

799r
BASE - LN
2.7712812921102037
VMAGANGLE - LN POLAR
[2739.8309024712335, -1.3121647314039884, 2739.1278224874604, -121.29882502851235, 2739.1278266691697, 118.70117520842838]
puVMAGANGLE - LN POLAR
[0.9886513181723887, -1.3121647314039884, 0.988397

In [42]:
dss.run_command('Redirect feeder/03node_multiphase_balanced.dss')

dss.Solution.Solve()
if not dss.Solution.Converged:
    print('Initial Solution Not Converged. Check Model for Convergence')
else:
    print('Initial Model Converged. Proceeding to Next Step.')
    #Doing this solve command is required for GridPV, that is why the monitors
    #go under a reset process
    dss.Monitors.ResetAll()
    
    #set solution Params
    #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
    dss.Solution.Mode(1)
    dss.Solution.Number(1)
    dss.Solution.StepSize(1)
    dss.Solution.ControlMode(-1)
    dss.Solution.MaxControlIterations(1000000)
    dss.Solution.MaxIterations(30000)
    #Easy process to get all names and count of loads, a trick to avoid
    #some more lines of code
    TotalLoads=dss.Loads.Count()
    AllLoadNames=dss.Loads.AllNames()
    print('OpenDSS Model Compliation Done.')

print('~~~~~')

for k1 in range(len(dss.RegControls.AllNames())):
    print(dss.RegControls.AllNames()[k1])
    dss.RegControls.Name(dss.RegControls.AllNames()[k1])
    
    print(dss.RegControls.TapNumber())
    
for k1 in range(len(dss.Circuit.AllBusNames())):
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
    print(dss.Circuit.AllBusNames()[k1])
    print(dss.Bus.puVmagAngle()[::2])
#     print('')

dss.run_command('Redirect feeder/03node_multiphase_balanced.dss')


# print(dss.Loads.AllNames())
dss.Loads.Name('s701a')
dss.Loads.kvar(-500)

dss.Loads.Name('s702a')
dss.Loads.kvar(-500)

dss.Solution.Solve()
if not dss.Solution.Converged:
    print('Initial Solution Not Converged. Check Model for Convergence')
else:
    print('Initial Model Converged. Proceeding to Next Step.')
    #Doing this solve command is required for GridPV, that is why the monitors
    #go under a reset process
    dss.Monitors.ResetAll()
    
    #set solution Params
    #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
    dss.Solution.Mode(1)
    dss.Solution.Number(1)
    dss.Solution.StepSize(1)
    dss.Solution.ControlMode(-1)
    dss.Solution.MaxControlIterations(1000000)
    dss.Solution.MaxIterations(30000)
    #Easy process to get all names and count of loads, a trick to avoid
    #some more lines of code
    TotalLoads=dss.Loads.Count()
    AllLoadNames=dss.Loads.AllNames()
    print('OpenDSS Model Compliation Done.')

print('~~~~~')

for k1 in range(len(dss.RegControls.AllNames())):
    print(dss.RegControls.AllNames()[k1])
    dss.RegControls.Name(dss.RegControls.AllNames()[k1])
    
    print(dss.RegControls.TapNumber())
    
for k1 in range(len(dss.Circuit.AllBusNames())):
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
    print(dss.Circuit.AllBusNames()[k1])
    print(dss.Bus.puVmagAngle()[::2])
#     print('')

dss.run_command('Redirect feeder/03node_multiphase_balanced.dss')

dss.Loads.Name('s701a')
dss.Loads.kvar(500)

dss.Loads.Name('s702a')
dss.Loads.kvar(500)

for k1 in range(len(dss.RegControls.AllNames())):
    print(dss.RegControls.AllNames()[k1])
    dss.RegControls.Name(dss.RegControls.AllNames()[k1])
    dss.RegControls.Reset()

dss.Solution.Solve()
if not dss.Solution.Converged:
    print('Initial Solution Not Converged. Check Model for Convergence')
else:
    print('Initial Model Converged. Proceeding to Next Step.')
    #Doing this solve command is required for GridPV, that is why the monitors
    #go under a reset process
    dss.Monitors.ResetAll()
    
    #set solution Params
    #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
    dss.Solution.Mode(1)
    dss.Solution.Number(1)
    dss.Solution.StepSize(1)
    dss.Solution.ControlMode(-1)
    dss.Solution.MaxControlIterations(1000000)
    dss.Solution.MaxIterations(30000)
    #Easy process to get all names and count of loads, a trick to avoid
    #some more lines of code
    TotalLoads=dss.Loads.Count()
    AllLoadNames=dss.Loads.AllNames()
    print('OpenDSS Model Compliation Done.')

print('~~~~~')

for k1 in range(len(dss.RegControls.AllNames())):
    print(dss.RegControls.AllNames()[k1])
    dss.RegControls.Name(dss.RegControls.AllNames()[k1])
    
    print(dss.RegControls.TapNumber())
    
for k1 in range(len(dss.Circuit.AllBusNames())):
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
    print(dss.Circuit.AllBusNames()[k1])
    print(dss.Bus.puVmagAngle()[::2])
#     print('')

Initial Model Converged. Proceeding to Next Step.
OpenDSS Model Compliation Done.
~~~~~
reg1
1
sourcebus
[0.9999982149564858, 0.9999981799427007, 0.999998181497053]
799
[0.9885057538096906, 0.9882484465843003, 0.988248448116674]
799r
[0.994671513122048, 0.9944125379742557, 0.994412539516184]
701
[0.9906864748845824, 0.9876077017531766, 0.9876077032824417]
702
[0.9862362349033518, 0.9831740093034612, 0.9831740108245071]
Initial Model Converged. Proceeding to Next Step.
OpenDSS Model Compliation Done.
~~~~~
reg1
-4
sourcebus
[1.0000034408028204, 0.9999984813279755, 0.9999984302002922]
799
[1.0252422216080284, 0.9890442160416161, 0.9890441655921566]
799r
[0.9996140883043692, 0.9643068348139464, 0.9643067856264155]
701
[1.0006444735381514, 0.9577492315247855, 0.9577491827411957]
702
[1.0096063201730894, 0.953476017734255, 0.9534759692128688]
reg1
Initial Model Converged. Proceeding to Next Step.
OpenDSS Model Compliation Done.
~~~~~
reg1
6
sourcebus
[0.999994153455065, 0.9999979105179184, 